In [ ]:
#CELL TO CONNECT GOOGLE DRIVE TO COLAB

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#Install ClearML, Ultralytics and import Yolo

!pip install ultralytics
!pip install clearml
import os
from ultralytics import YOLO


In [ ]:
#FINETUNING AND CONFIGURATION TO MAKE EXPERIMENT  EASY TO MANAGE: https://www.kaggle.com/code/hinepo/yolov8-finetuning-for-ppe-detection

class CFG:
    DEBUG = False
    FRACTION = 0.05 if DEBUG else 1.0
    SEED = 88
    # training
    EPOCHS = 3 if DEBUG else 300 # 100
    BATCH_SIZE = 16

    BASE_MODEL = 'yolov8l' # yolov8n, yolov8s, yolov8m, yolov8l, yolov8x
    BASE_MODEL_WEIGHTS = f'{BASE_MODEL}.pt'

    #train directory and ClearML tracking, manually edit  epochs if resume training.
    EXP_NAME = f'Experiment-11_PPE_Detector_{BASE_MODEL}_{EPOCHS}_epochs'

    CONTINUE_LAST_TASK = False  #Or task ID (if you want continue a previous experiment)

    OPTIMIZER = 'auto' # SGD, Adam, Adamax, AdamW, NAdam, RAdam, RMSProp, auto
    LR = 1e-3
    LR_FACTOR = 0.01
    WEIGHT_DECAY = 5e-4
    DROPOUT = 0.0
    PATIENCE = 20
    PROFILE = False
    LABEL_SMOOTHING = 0.0


    # paths
    ROOT_DIR = '/content/gdrive/My Drive/PPE Detection/' #WRITE YOU ROOT DIRECTORY HERE




In [ ]:
#Configure and start ClearML for experiment tracking

from clearml import Task, Logger
Task.set_credentials(
    api_host = "https://api.clear.ml",
    web_host = "https://app.clear.ml",
    files_host = "https://files.clear.ml",
    key = 'key', # Your ClearML key
    secret =  'secret' #Your ClearML secret key
)
task = Task.init(project_name = 'PPE Detection', task_name = CFG.EXP_NAME, continue_last_task = CFG.CONTINUE_LAST_TASK)

In [ ]:
# Train model

#Load model
model = YOLO(CFG.BASE_MODEL_WEIGHTS)

#Use model
model.train(
    data=os.path.join(CFG.ROOT_DIR, "data.yaml"),
    task = 'detect',
    epochs = CFG.EPOCHS,
    batch = CFG.BATCH_SIZE,
    optimizer = CFG.OPTIMIZER,
    lr0 = CFG.LR,
    lrf = CFG.LR_FACTOR,
    weight_decay = CFG.WEIGHT_DECAY,
    dropout = CFG.DROPOUT,
    fraction = CFG.FRACTION,
    patience = CFG.PATIENCE,
    profile = CFG.PROFILE,
    label_smoothing = CFG.LABEL_SMOOTHING,

    name = f'{CFG.EXP_NAME}',
    seed = CFG.SEED,
    val = True,
    amp = True,
    exist_ok = True,
    resume = False,
    device = 0,
    verbose = False,
    )

#SAVE OUTPUT TO GOOGLE DRIVE
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install aspose-words
!scp -r /content/runs/ '/content/gdrive/My Drive/PPE Detection/' #adjust it manually if don't work


In [ ]:
#Get weights to make inference

model = YOLO("/content/gdrive/MyDrive/PPE Detection/runs/detect/PPE_Detector_yolov8l_finetuning-133_epochs/weights/best.pt")

In [ ]:
# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

In [ ]:
#Make inference on a video

!yolo detect predict model = "/content/gdrive/MyDrive/PPE Detection/runs/detect/PPE_Detector_yolov8l_finetuning-133_epochs/weights/best.pt"  source = "/content/gdrive/MyDrive/PPE Detection/data/videos/workers-video-1.mp4"

In [ ]:
#Save the video on runs/

!scp -r /content/runs '/content/gdrive/My Drive/PPE Detection/'

In [ ]:
#Convert the video to mp4 format
!ffmpeg -i {"/content/gdrive/MyDrive/PPE Detection/runs/detect/predict/workers-video-2.avi"} -vcodec libx264 {"yolov8l-ppe-detection-video-1.mp4"}